In [2]:
import spacy 
import numpy as np
import itertools
nlp = spacy.load("en_core_web_md")


In [74]:
feature = ["Meet with HR at 12:30pm"]
labels = ["T", "T", "T", "T", "O", "O"]

feature_pos = []
tokens = nlp(feature[0])
for token in tokens:
    item = (token.text, token.pos_)
    feature_pos.append(item)
    print(item)

('Meet', 'VERB')
('with', 'ADP')
('HR', 'PROPN')
('at', 'ADP')
('12:30pm', 'NOUN')


In [27]:
def get_labels(values, n):
    sequences = list(itertools.product(values, repeat=n))
    return sequences

values = ["T", "O"]
n = 3
sequences = get_labels(values, n)
for seq in sequences:
    print(seq)

('T', 'T', 'T')
('T', 'T', 'O')
('T', 'O', 'T')
('T', 'O', 'O')
('O', 'T', 'T')
('O', 'T', 'O')
('O', 'O', 'T')
('O', 'O', 'O')


In [ ]:
def f1(tags, labels):
    if len(tags) != len(labels):
        raise ValueError("Tags and labels must have the same length")
    
    for i, tag in enumerate(tags):
        if tag == "ADJ" and i > 0:
            if labels[i] == "T" and (tags[i - 1] == "DET" or tags[i - 1] == "VERB") and labels[i - 1] == "T":
                return 1
    return 0

def f2(tags, labels):
    if len(tags) != len(labels):
        raise ValueError("Tags and labels must have the same length")
    
    for i, tag in (enumerate(tags)):
        if tag == "NOUN" and i > 0:
            if labels[i] == "T" and tags[i - 1] == "ADJ" and labels[i - 1] == "T":
                return 1
    return 0

def f3(tags, labels):
    if len(tags) != len(labels):
        raise ValueError("Tags and labels must have the same length")
    
    for i, tag in (enumerate(tags)):
        if tag == "VERB" and  i < len(tags) - 1:
            if labels[i] == "T" and (tags[i+1] == "NOUN" or tags[i+1] == "ADJ"):
                return 1
    return 0

sequences = set(get_labels(labels, len(labels)))
tags = [item[1] for item in feature_pos]
features = [item[0] for item in feature_pos]
print(f"Features: {features}")
print(f"Tags: {tags}")
weight1 = np.random.rand(1)
weight2 = np.random.rand(1)
weight3 = np.random.rand(1)

total = 0
for seq in sequences:
    output1 = f1(tags, seq) * weight1
    output2 = f2(tags, seq) * weight2
    output3 = f3(tags, seq) * weight3
    
    weighted_sum = output1 + output2 + output3
    total += weighted_sum

true1 = f1(tags, labels) * weight1
true2 = f2(tags, labels) * weight2
true3 = f3(tags, labels) * weight3
true_weighted_sum = true1 + true2 + true3
probability = true_weighted_sum / total
loss = np.log(total) - true_weighted_sum
print(f"Probability of the sequence being correct: {probability}")
print(f"Loss to minimize: {float(loss)}")


Features: ['Submit', 'the', 'quarterly', 'report', 'by', '6:30am']
Tags: ['VERB', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN']
Probability of the sequence being correct: [0.0625]
Loss to minimize: [1.6028191]


In [64]:
def loss(true_probability):
    return -np.log(true_probability)

def z(true_score, sequences, tags, weights):
    total = 0
    for seq in sequences:
        output1 = np.exp(f1(tags, seq) * weights[0])
        output2 = np.exp(f2(tags, seq) * weights[1])
        output3 = np.exp(f3(tags, seq) * weights[2])
        
        weighted_sum = output1 + output2 + output3
        total += weighted_sum

    return np.exp(true_score) / total

# ∑​P(y'∣x) * f(x,y') - f(x, y_true) 
def gradient(true_scores, tags, sequences, weights, learning_rate=0.01):
    z_score = z(np.sum(true_scores), sequences, tags, weights) 
    for seq in sequences:
        output1 = f1(tags, seq) * weights[0]
        gradient1 = (z_score * output1) - true_scores[0]
        weights[0] -= learning_rate * gradient1

        output2 = f2(tags, seq) * weights[1]
        gradient2 = (z_score * output2) - true_scores[1]
        weights[1] -= learning_rate * gradient2

        output3 = f3(tags, seq) * weights[2]
        gradient3 = (z_score * output3) - true_scores[2]
        weights[2] -= learning_rate * gradient3

    return weights

true_scores = [true1, true2, true3]
weights = [weight1, weight2, weight3]
print(f"Initial weights: {weights}")

new_weights = gradient(true_scores, tags, sequences, weights)
print(f"Updated weights: {new_weights}")
print(f"Loss after gradient update: {loss(np.sum(true_scores))}")

total = 0
for seq in sequences:
    output1 = f1(tags, seq) * new_weights[0]
    output2 = f2(tags, seq) * new_weights[1]
    output3 = f3(tags, seq) * new_weights[2]
    
    weighted_sum = output1 + output2 + output3
    total += weighted_sum

probability = np.sum(true_scores) / total
print(f"Probability of the sequence being correct after update: {probability}")


Initial weights: [array([0.38705005]), array([0.47229748]), array([0.36514545])]
Updated weights: [array([0.53767342]), array([0.6560778]), array([0.36514545])]
Loss after gradient update: 0.6452252379776657
Probability of the sequence being correct after update: [0.02746303]
